In [1]:
%load_ext autoreload
from coffea import util, processor
from msdprocessor5 import msdProcessor

from coffea.nanoevents import NanoEventsFactory, PFNanoAODSchema
import json
import distributed
import dask
import awkward as ak
import hist
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.animation import FuncAnimation, PillowWriter
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
from hist import Hist
import dask_awkward
import os
import math

In [2]:
directory_path = "/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/"

fileset = []
for filename in os.listdir(directory_path):
    if filename.endswith(".root"):
        # Add the full file path to the fileset list
        fileset.append(os.path.join(directory_path, filename))

print("Fileset:", fileset)

Fileset: ['/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/ggF.root', '/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/VBF.root', '/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/WminusH.root', '/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/ZH.root', '/uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/QCD_Pt470to600.root']


In [3]:
n_global = 1
beta_range_global = 0  # For instance, beta=0 if n=1
z_cut_range_global = 1 # For instance, z_cut=1 if n=1

def trunc(num):
    """Truncate to 3 decimal places."""
    return math.trunc(num * 1000) / 1000

In [4]:
for file in fileset:
    # Extract the production mode name from the file path
    prod_mode = file.split('/data-mc/')[-1].replace('.root', '')
    print(f"\nProcessing file:  {file}")
    print(f"Production mode:  {prod_mode}")

    # Read events using NanoEventsFactory
    events = NanoEventsFactory.from_root(
        {file: "/Events"},
        schemaclass=PFNanoAODSchema
    ).events()


Processing file:  /uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/ggF.root
Production mode:  ggF


/uscms/home/tbabenko/nobackup/miniconda3/envs/fastjet/lib/python3.12/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector



Processing file:  /uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/VBF.root
Production mode:  VBF

Processing file:  /uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/WminusH.root
Production mode:  WminusH

Processing file:  /uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/ZH.root
Production mode:  ZH

Processing file:  /uscms/home/jennetd/nobackup/hbb-prod-modes/run3-triggers/data-mc/QCD_Pt470to600.root
Production mode:  QCD_Pt470to600


In [5]:
result = msdProcessor().process(
        events,
        beta=beta_range_global,
        z_cut=z_cut_range_global,
        n=n_global
    )

In [7]:
compute = dask.compute(result)
print(f"Result for {prod_mode}:", compute)


#--------------------------------------------------------------------------
#                         FastJet release 3.4.1
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
Result for QCD_Pt470to600: ([{'sumw': 87635.195, 'b00': Hist(Regular(40

In [9]:
n_betas = n_global
n_zcuts = n_betas

compute_matrix = [[None for _ in range(n_zcuts)] for _ in range(n_betas)]
for beta in range(n_betas):
    for z_cut in range(n_zcuts):
        # Access the histogram stored under f"b{beta}{z_cut}"
        compute_matrix[beta][z_cut] = compute[0][0][f"b{beta}{z_cut}"]

In [10]:
for z_cut in range(n_zcuts):
        for beta in range(n_betas):
            current_beta = trunc(beta * (beta_range_global) / n_global)
            current_z_cut = trunc(z_cut * (z_cut_range_global) / n_global)

            # Construct a filename that includes prod_mode, beta, and z_cut
            coffea_filename = f"{prod_mode}_beta{current_beta}_zcut{current_z_cut}.coffea"

            # Save this single histogram data to the Coffea file
            util.save(compute_matrix[beta][z_cut], coffea_filename)
            print(f"Saved Coffea file: {coffea_filename}")


Saved Coffea file: QCD_Pt470to600_beta0.0_zcut0.0.coffea


In [ ]:
x_min, x_max = 0, 400
y_min, y_max = 0, 800

for z_cut in range(n_zcuts):
    for beta in range(n_betas):
        # Create the figure
        fig, ax = plt.subplots()

        current_beta = trunc(beta * (beta_range_global) / n_global)
        current_z_cut = trunc(z_cut * (z_cut_range_global) / n_global)

        # Plot the histogram
        compute_matrix[beta][z_cut].plot1d(ax=ax, 
            label=f"beta = {current_beta}, z_cut = {current_z_cut}")

        # Fix axis limits
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
    
        # Set a title
        ax.set_title(f"File = {prod_mode}, beta = {current_beta}, z_cut = {current_z_cut}")
    
        # Add legend and save the figure
        ax.legend()
        plot_filename = f"{prod_mode}-zcut{current_z_cut}-beta{current_beta}.png"
        plt.savefig(plot_filename, dpi=300)
        plt.close(fig)

        print(f"Saved plot: {plot_filename}")